- [ ] Create product subdir tree 
- [ ] Download all files to subdirs
    - [ ] List all files
    - [ ] Parse location from safe dir 
    - [ ] move all files to their local dir
- [ ] ID burst overlapping with AOI
    - [ ] Use burst API? Get IDs and products
    - [ ] Or use burst info in product
- [ ] Compute burst offsets
- [ ] Open datasets and add a burst_start and burst_end attribute
- [ ] Download the burst range in each IW to zarr

In [ ]:
from pystac_client.client import Client
import geopandas as gpd
import rioxarray as riox
import xarray as xr
import json
from dask.diagnostics import ProgressBar
from pathlib import Path
from urllib.parse import urlparse
import os

from rasterio.session import AWSSession
import rasterio
import boto3

In [ ]:
out_dir = "/data/res/test_partial_product/"
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

# use the creds created on CDSE website
with open("/data/creds_s3.json") as f:
    cred = json.load(f)

un = cred["username"]
pw = cred["password"]

# rasterio handles S3 sessions
rio_session = AWSSession(
    aws_access_key_id=un,
    aws_secret_access_key=pw,
    region_name="default",
    endpoint_url="eodata.dataspace.copernicus.eu")


session = boto3.session.Session()
s3 = boto3.resource(
    's3',
    endpoint_url='https://eodata.dataspace.copernicus.eu',
    aws_access_key_id=un,
    aws_secret_access_key=pw,
    region_name='default'
)

# aws_session = boto3.Session(
#     aws_access_key_id=un,
#     aws_secret_access_key=pw,
#     region_name="default",
# )

In [49]:
# Search using STAC api
catalog = Client.open(
    "https://stac.dataspace.copernicus.eu/v1/"
)
aoi_file = "../data/Morocco_AOI.geojson"
shp = gpd.read_file(aoi_file).geometry[0]
search = catalog.search(collections=["sentinel-1-slc"], intersects=shp)
it = next(search.items())
product_id = it.id
product_root_dir = f"{product_id}.SAFE"
product_root_dir

url = it.assets["iw1-vv"].href

manifest_url = it.assets["safe_manifest"].href


# Parse the URL
parsed = urlparse(manifest_url)

# Bucket is the "netloc"
bucket_name = parsed.netloc
manifest_path = Path(parsed.path.lstrip("/"))
prefix = str(manifest_path.parent)

In [ ]:
# open remote dataset
with rasterio.Env(session=rio_session, AWS_VIRTUAL_HOSTING=False):
    ds = riox.open_rasterio(url, chunks="auto")

In [ ]:
# download cropped array
with ProgressBar():
    ds[0, :1500].to_zarr("/data/res/test_s3_S1.zarr", mode="w")

In [ ]:
url2 = it.assets["schema-product-iw1-vh"]
url2

In [40]:
prefix.anchor

'/'

In [ ]:
# try to find xml files
bucket = s3.Bucket(bucket_name)
files = [it.key for it in list(
    bucket.objects.filter(
        Prefix=prefix
    )
)]


In [51]:
# f = files[0].key
# Path(f).parent.name
for f in files:
    print(Path(f))
subdirs = ("annotation", "measurement", "preview", "support", "annotation/rfi")


Sentinel-1/SAR/IW_SLC__1S/2025/09/30/S1C_IW_SLC__1SDV_20250930T062813_20250930T062840_004350_0089FE_0756.SAFE/S1C_IW_SLC__1SDV_20250930T062813_20250930T062840_004350_0089FE_0756.SAFE-report-20250930T070344.pdf
Sentinel-1/SAR/IW_SLC__1S/2025/09/30/S1C_IW_SLC__1SDV_20250930T062813_20250930T062840_004350_0089FE_0756.SAFE/annotation/calibration/calibration-s1c-iw1-slc-vh-20250930t062813-20250930t062838-004350-0089fe-001.xml
Sentinel-1/SAR/IW_SLC__1S/2025/09/30/S1C_IW_SLC__1SDV_20250930T062813_20250930T062840_004350_0089FE_0756.SAFE/annotation/calibration/calibration-s1c-iw1-slc-vv-20250930t062813-20250930t062838-004350-0089fe-004.xml
Sentinel-1/SAR/IW_SLC__1S/2025/09/30/S1C_IW_SLC__1SDV_20250930T062813_20250930T062840_004350_0089FE_0756.SAFE/annotation/calibration/calibration-s1c-iw2-slc-vh-20250930t062814-20250930t062839-004350-0089fe-002.xml
Sentinel-1/SAR/IW_SLC__1S/2025/09/30/S1C_IW_SLC__1SDV_20250930T062813_20250930T062840_004350_0089FE_0756.SAFE/annotation/calibration/calibration-s1c

This fails:
```bash
./sentinel1_burst_extractor_spatiotemporal.sh -p vv -s 2025-09-25 -e 2025-09-30 -x 30.19358 -y -7.2473372
```
Example from the docs also fails. Problem with string parsing.